<a href="https://colab.research.google.com/github/VIJAYARAGUL362/SPACE-TITANIC-CHALLENGE/blob/main/SPACESHIP_TITANIC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IMPORTING THE LIBRARIES

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# IMPORTING THE DATASET

## GETTING THE DATA FROM KAGGLE

In [ ]:
# GETTING THE KAGGLE API KEY
from google.colab import files
files.upload()

In [ ]:
# INSTALLING KAGGLE
!pip install kaggle

In [ ]:
# Create the .kaggle directory if it doesn't exist
!mkdir -p ~/.kaggle/

# Move the uploaded kaggle.json to the .kaggle directory
!mv kaggle.json ~/.kaggle/

# Set read/write permissions for the owner only (secure)
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# DOWNLOADING THE DATASET
!kaggle competitions download -c spaceship-titanic

In [ ]:
import os
import zipfile

# Replace 'your_dataset_name.zip' with the actual name of your downloaded zip file
zip_file_name = 'spaceship-titanic.zip' # Example for the Titanic dataset

if os.path.exists(zip_file_name):
    with zipfile.ZipFile(zip_file_name, 'r') as zip_ref:
        zip_ref.extractall('.') # Extracts all contents to the current directory
    print(f"'{zip_file_name}' unzipped successfully.")
else:
    print(f"Error: '{zip_file_name}' not found. Check the filename.")

# Optional: List files again to see the unzipped contents
!ls

## LOADING THE DATASET

In [ ]:
train_dataset = pd.read_csv('train.csv')
train_dataframe = pd.DataFrame(train_dataset)

In [ ]:
train_dataframe

In [ ]:
train_dataframe.info()

In [ ]:
# SEGREGATING THE DATASET
categorical_feature = train_dataframe.select_dtypes('object')
numerical_feature = train_dataframe.select_dtypes('float')
target_feature = train_dataframe.select_dtypes('bool')

In [ ]:
categorical_feature.drop(columns=["PassengerId","Name"],inplace=True,axis=1)

In [ ]:
categorical_feature

In [ ]:
numerical_feature

# DATA PROCESSING

## CATEGORICAL VARIABLE

In [ ]:
# MISSING VALUES
categorical_feature['HomePlanet'].fillna('Earth',inplace=True)

In [ ]:
# MISSING VALUES
categorical_feature['CryoSleep'].fillna(False,inplace=True)

In [ ]:
# MISSING VALUES
categorical_feature['Cabin'].fillna("G",inplace=True)

In [ ]:
# DATA MANIPULATION
categorical_feature['Cabin'] = categorical_feature['Cabin'].apply(lambda x: x[0])

In [ ]:
index_id_cabin,unique_values_cabin = categorical_feature['Cabin'].factorize()

In [ ]:
index_id_cabin,unique_values_cabin

In [ ]:
categorical_feature['Cabin'] = index_id_cabin

In [ ]:
# MISSING VALUES
categorical_feature['Destination'].fillna("T",inplace=True)

In [ ]:
categorical_feature['Destination'].unique()

In [ ]:
# DATA MANIPULATION
index_id_dest,unique_values_dest = categorical_feature['Destination'].factorize()

In [ ]:
categorical_feature['Destination'] = index_id_dest

In [ ]:
# MISSING VALUES
categorical_feature['VIP'].fillna(False,inplace = True)

In [ ]:
categorical_feature.isna().sum()

### ONE HOT ENCODING THE CATEGORICAL DATASET

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
ct = ColumnTransformer(transformers=[('encoder',OneHotEncoder(drop="first"),['HomePlanet','CryoSleep','Cabin','Destination','VIP'])],remainder="passthrough")

In [ ]:
encoded_cat = ct.fit_transform(categorical_feature)

In [ ]:
encoded_cat

### CREATING A NEW CATEGORICAL DATAFRAME

In [ ]:
cat_column_names = list(ct.get_feature_names_out())

In [ ]:
new_cat_dataframe = pd.DataFrame(encoded_cat.toarray(),columns=cat_column_names)

In [ ]:
new_cat_dataframe

## NUMERICAL VARIABLE

In [ ]:
numerical_feature

In [ ]:
# DATAFRAME INFO
numerical_feature.info()

In [ ]:
numerical_feature.isna().sum()

In [ ]:
# MISSING VALUES
numerical_feature.fillna(numerical_feature['Age'].mean(),inplace=True)

In [ ]:
# MISSING VALUES
numerical_feature['RoomService'].fillna(numerical_feature['RoomService'].mean(),inplace=True)

In [ ]:
# MISSING VALUES
numerical_feature['ShoppingMall'].fillna(numerical_feature['ShoppingMall'].mean(),inplace=True)

In [ ]:
# MISSING VALUES
numerical_feature['Spa'].fillna(numerical_feature['Spa'].mean(),inplace=True)

In [ ]:
# MISSING VALUES
numerical_feature['VRDeck'].fillna(numerical_feature['VRDeck'].mean(),inplace=True)

In [ ]:
numerical_feature

In [ ]:
numerical_feature.isna().sum()

### SCALING THE NUMERICAL VARIABLE

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

In [ ]:
scaled_num = sc.fit_transform(numerical_feature)

### CREATING A NEW NUMERICAL DATAFRAME

In [ ]:
num_column_names = sc.get_feature_names_out()
num_column_names

In [ ]:
new_num_dataframe = pd.DataFrame(scaled_num,columns=num_column_names)

In [ ]:
new_num_dataframe

# CREATING A TRAINING DATAFRAME

In [ ]:
training_dataframe = pd.concat([new_cat_dataframe,new_num_dataframe],axis=1)

In [ ]:
training_dataframe

# TARGET VARIABLE

In [ ]:
index_target,unique_values_target = target_feature['Transported'].factorize()

In [ ]:
index_target,unique_values_target

In [ ]:
target_feature['Transported'] = index_target

In [ ]:
target_feature

# SPLITTING THE DATASET

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(training_dataframe,target_feature,test_size=0.2)

In [ ]:
X_train.shape,X_test.shape

In [ ]:
y_train.shape,y_test.shape

# CREATING THE MODEL

In [ ]:
from sklearn.svm import SVC
svc = SVC(kernel="rbf")

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=300)

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier()

# TUNING THE MODEL

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
# RANDOM FOREST
param_grid = {
    'n_estimators': [100, 200],  # Number of trees
    'max_depth': [10, 20, None],  # Maximum depth of trees
    'min_samples_split': [2, 5],  # Minimum samples to split a node
    'min_samples_leaf': [1, 2],   # Minimum samples at a leaf
    'max_features': ['sqrt', 'log2']  # Number of features to consider at each split
}

tuned_rfc = GridSearchCV(
    estimator=rfc,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    scoring='f1_macro',  # Optimize for F1-score (macro average)
    n_jobs=-1,  # Use all available CPU cores
    verbose=1
)

# TRAINING THE MODEL

In [ ]:
svc.fit(X_train,y_train)

In [ ]:
rfc.fit(X_train,y_train)

In [ ]:
lr.fit(X_train,y_train)

In [ ]:
dtc.fit(X_train,y_train)

In [ ]:
# tuned_rfc.fit(X_train,y_train)

# EVALUATING THE MODEL

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
y_pred_svc = svc.predict(X_test)
score_svc = classification_report(y_test,y_pred_svc)
score_svc

In [ ]:
y_pred_rfc = rfc.predict(X_test)
score_rfc = classification_report(y_test,y_pred_rfc)
score_rfc

In [ ]:
y_pred_lr = lr.predict(X_test)
score_lr = classification_report(y_test,y_pred_lr)
score_lr

In [ ]:
y_pred_dtc = dtc.predict(X_test)
score_dtc = classification_report(y_test,y_pred_dtc)
score_dtc

In [ ]:
# y_pred_trfc = tuned_rfc.predict(X_test)
# score_t_rfc = classification_report(y_test,y_pred_dtc)
# score_t_rfc

# DEEP LEARNING

## IMPORTING LIBRARIES

In [ ]:
import torch
from torch import nn

## DEVICE AGNOSTIC CODE

In [ ]:
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

In [ ]:
device

## MODEL ARCHIETECTURE

In [ ]:
class classification_model(nn.Module):
    def __init__(self) -> None:
        super().__init__()

        self.layers = nn.Sequential(
            nn.Linear(in_features=20,out_features=32),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(in_features=32,out_features=64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(in_features=64,out_features=1)
        )

    def forward(self,x):
        return self.layers(x)

In [ ]:
model = classification_model()

## OPTIMIZER AND COST FUNCTION

In [ ]:
loss = nn.BCEWithLogitsLoss()

optimizer = torch.optim.Adam(model.parameters(), lr=0.1)

## MOVING TO DATSET TO THE TENSORS

In [ ]:
X_train = torch.tensor(X_train.values,dtype=torch.float).to(device)
X_test = torch.tensor(X_test.values,dtype=torch.float).to(device)
y_train = torch.tensor(y_train.values,dtype=torch.float).to(device)
y_test = torch.tensor(y_test.values,dtype=torch.float).to(device)

In [ ]:
X_train.shape,X_test.shape

## TRAINING THE MODEL

In [ ]:
epochs = 300

training_loss = []
testing_loss = []
epoch_list = []

for epoch in range(epochs):

    # TRAINING THE MODE
    model.train()

    # FORWARD PROPAGATION
    y_train_pred = model(X_train)
    train_loss = loss(y_train_pred,y_train)

    # OPTIMIZER ZERO GRAD
    optimizer.zero_grad()

    # BACK PROPOGATION
    train_loss.backward()

    # GRADIENT DESCENT
    optimizer.step()

    # EVALUATING THE MODEL
    model.eval()
    with torch.inference_mode():
        y_test_pred = model(X_test)
        test_loss = loss(y_test_pred,y_test)

        if epoch%100 == 0:
            training_loss.append(train_loss)
            testing_loss.append(test_loss)
            print(f"Epoch is {epoch}:The training loss is {train_loss} and the test loss is {test_loss}")

In [ ]:
# MODEL PREDICTION
with torch.inference_mode():
    y_pred_logits = model(X_test)
    sigmoid = nn.Sigmoid()
    y_pred_nn = torch.round(sigmoid(y_pred_logits))

In [ ]:
y_pred_nn

In [ ]:
score_nn = classification_report(y_test,y_pred_nn)

In [ ]:
score_nn

# TEST DATASET

## IMPORTING THE DATASET

In [ ]:
test_dataset = pd.read_csv('test.csv')
test_dataframe = pd.DataFrame(test_dataset)

In [ ]:
test_dataframe

In [ ]:
test_dataframe.info()

In [ ]:
# SEGREGATING THE DATASET
categorical_feature_test = test_dataframe.select_dtypes('object')
numerical_feature_test = test_dataframe.select_dtypes('float')

## DATA PRE PROCESSING

### CATEGORICAL FEATURE

In [ ]:
# MISSING VALUES
categorical_feature_test['Cabin'].fillna("G",inplace=True)
categorical_feature_test['HomePlanet'].fillna('Earth',inplace=True)
categorical_feature_test['CryoSleep'].fillna(False,inplace=True)
categorical_feature_test['Cabin'] = categorical_feature_test['Cabin'].apply(lambda x: x[0])

In [ ]:
categorical_feature_test['VIP'].fillna(False,inplace = True)

In [ ]:
unique_values_cabin

In [ ]:
unique_values_dest

In [ ]:
categorical_feature_test['Destination'] = categorical_feature_test['Destination'].fillna("TRAPPIST-1e")

In [ ]:
categorical_feature_test

In [ ]:
test_passenger_id = pd.DataFrame(categorical_feature_test['PassengerId'],columns=["PassengerId"])
categorical_feature_test.drop(["PassengerId","Name"],inplace=True,axis=1)

In [ ]:
categorical_feature_test['Cabin'] = categorical_feature_test['Cabin'].apply(lambda x: list(unique_values_cabin).index(x))

In [ ]:
categorical_feature_test['Destination'] = categorical_feature_test['Destination'].apply(lambda x: list(unique_values_dest).index(x))

In [ ]:
categorical_feature_test

In [ ]:
## SCALING THE DATASET
enc_test_cat_dataset_sparse = ct.transform(categorical_feature_test)

In [ ]:
enc_cat_test = pd.DataFrame(enc_test_cat_dataset_sparse.toarray(),columns=ct.get_feature_names_out())

In [ ]:
enc_cat_test

### NUMERICAL DATASET

In [ ]:
numerical_feature_test

In [ ]:
numerical_feature_test.info()

In [ ]:
numerical_feature_test.isna().sum()

In [ ]:
# MISSING VALUES
numerical_feature_test.fillna(numerical_feature_test['Age'].mean(),inplace=True)
numerical_feature_test['RoomService'].fillna(numerical_feature_test['RoomService'].mean(),inplace=True)
numerical_feature_test['ShoppingMall'].fillna(numerical_feature_test['ShoppingMall'].mean(),inplace=True)
numerical_feature_test['Spa'].fillna(numerical_feature_test['Spa'].mean(),inplace=True)
numerical_feature_test['VRDeck'].fillna(numerical_feature_test['VRDeck'].mean(),inplace=True)

In [ ]:
numerical_feature_test

In [ ]:
scaled_test_num_array = sc.transform(numerical_feature_test)

In [ ]:
scaled_test_num = pd.DataFrame(scaled_test_num_array,columns=sc.get_feature_names_out())

In [ ]:
scaled_test_num

### COMBINING THE BOTH THE FEATURES

In [ ]:
new_test_dataset = pd.concat([enc_cat_test,scaled_test_num],axis=1)

In [ ]:
new_test_dataset

## INFERENCE ON THE TEST DATASET

In [ ]:
# MODEL PREDICTION
tensor_test_dataset = torch.tensor(new_test_dataset.values,dtype=torch.float).to(device)
with torch.inference_mode():
    y_pred_logits_test = model(tensor_test_dataset)
    sigmoid = nn.Sigmoid()
    y_pred_nn_test = torch.round(sigmoid(y_pred_logits_test))

In [ ]:
y_pred_nn_test

In [ ]:
test_passenger_id['Transported'] = y_pred_nn_test

In [ ]:
test_passenger_id['Transported'] = test_passenger_id['Transported'].apply(lambda x:True if x == 1 else False)

In [ ]:
test_passenger_id.to_csv('NEUREL_NETWORK_PREDICTION.csv',index=False)

# TRAINING THE MODELS ON WHOLE DATASET

In [ ]:
model2 = classification_model()

In [ ]:
loss1 = nn.BCEWithLogitsLoss()

optimizer1 = torch.optim.Adam(model2.parameters(), lr=0.1)

In [ ]:
epochs = 300

training_loss = []
testing_loss = []
epoch_list = []

torch_training_dataset = torch.tensor(training_dataframe.values,dtype=torch.float)
torch_target_variable = torch.tensor(target_feature.values,dtype=torch.float)


for epoch in range(epochs):

    # TRAINING THE MODE
    model2.train()

    # FORWARD PROPAGATION
    y_train_pred = model2(torch_training_dataset)
    train_loss = loss1(y_train_pred,torch_target_variable)

    # OPTIMIZER ZERO GRAD
    optimizer1.zero_grad()

    # BACK PROPOGATION
    train_loss.backward()

    # GRADIENT DESCENT
    optimizer1.step()

    # EVALUATING THE MODEL
    # model.eval()
    # with torch.inference_mode():
    #     y_test_pred = model(X_test)
    #     test_loss = loss(y_test_pred,y_test)

    #     if epoch%100 == 0:
    #         training_loss.append(train_loss)
    #         testing_loss.append(test_loss)
    #         print(f"Epoch is {epoch}:The training loss is {train_loss} and the test loss is {test_loss}")

In [ ]:
# MODEL PREDICTION
tensor_test_dataset = torch.tensor(new_test_dataset.values,dtype=torch.float).to(device)
with torch.inference_mode():
    y_pred_logits_test = model2(tensor_test_dataset)
    sigmoid = nn.Sigmoid()
    y_pred_nn_test = torch.round(sigmoid(y_pred_logits_test))

In [ ]:
test_passenger_id['Transported'] = y_pred_nn_test

In [ ]:
test_passenger_id['Transported'] = test_passenger_id['Transported'].apply(lambda x:True if x == 1 else False)

In [ ]:
test_passenger_id.to_csv('NEUREL_NETWORK_PREDICTION.csv',index=False)